In [67]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

In [68]:
features = np.asarray(np.load("/home/sorozco0612/dev/alan_bot/data_processing/road_data/features.npy"))
labels = np.asarray(np.load("/home/sorozco0612/dev/alan_bot/data_processing/road_data/labels.npy"))
features = features.reshape(list(features.shape)+ [1])

print(features.shape)
print(labels.shape)


x_train, x_test, y_train, y_test = train_test_split(features,labels)


(32, 33, 47, 1)
(32, 2)


In [69]:
model = models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation='relu', input_shape=(33,47,1)))
model.add(layers.MaxPool2D((2,2)))

model.add(layers.Flatten())
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(2, activation='sigmoid'))

model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 31, 45, 32)        320       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 15, 22, 32)        0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 10560)             0         
_________________________________________________________________
dense_16 (Dense)             (None, 64)                675904    
_________________________________________________________________
dense_17 (Dense)             (None, 2)                 130       
Total params: 676,354
Trainable params: 676,354
Non-trainable params: 0
_________________________________________________________________


In [70]:
model.compile(optimizer='adam',
              loss = tf.keras.losses.MeanSquaredError(),
              metrics=['mean_squared_error'])

In [71]:
fit = model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test))

Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 0.2500 - mean_squared_error: 0.2500 - val_loss: 0.2498 - val_mean_squared_error: 0.2498
Epoch 2/10
1/1 [==============================] - 0s 24ms/step - loss: 0.2498 - mean_squared_error: 0.2498 - val_loss: 0.2495 - val_mean_squared_error: 0.2495
Epoch 3/10
1/1 [==============================] - 0s 25ms/step - loss: 0.2495 - mean_squared_error: 0.2495 - val_loss: 0.2493 - val_mean_squared_error: 0.2493
Epoch 4/10
1/1 [==============================] - 0s 23ms/step - loss: 0.2493 - mean_squared_error: 0.2493 - val_loss: 0.2490 - val_mean_squared_error: 0.2490
Epoch 5/10
1/1 [==============================] - 0s 22ms/step - loss: 0.2490 - mean_squared_error: 0.2490 - val_loss: 0.2488 - val_mean_squared_error: 0.2488
Epoch 6/10
1/1 [==============================] - 0s 23ms/step - loss: 0.2488 - mean_squared_error: 0.2488 - val_loss: 0.2485 - val_mean_squared_error: 0.2485
Epoch 7/10
1/1 [==============================] 